# Lab | Inferential statistics

In [64]:
import pandas as pd
import numpy as np
import scipy.stats as st
import chardet

## T-tests

One-tailed t-test - In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on average than the machine currently used. To test that hypothesis, the times each machine takes to pack ten cartons are recorded. The results are in seconds in the tables in the file files_for_lab/machine.txt. Assume that there is sufficient evidence to conduct the t-test, does the data provide sufficient evidence to show if one machine is better than the other?

In [65]:
# Detecting file encoding with chardet

file_path = 'files_for_lab/machine.txt'

with open(file_path, 'rb') as file:
    result = chardet.detect(file.read())
    encoding = result['encoding']
    confidence = result['confidence']

print(f'The detected encoding is {encoding} with confidence {confidence:.2f}')

The detected encoding is UTF-16 with confidence 1.00


In [66]:
# Reading txt as dataframe

machine_data = pd.read_csv('files_for_lab/machine.txt', sep='\t', encoding='UTF16')
machine_data

,New machine,Old machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5
5,42.8,43.5
6,43.2,43.1
7,42.3,41.7
8,41.8,44.0
9,42.7,44.1


In [67]:
# Separating performance data from both machines

machine_new = machine_data.iloc[:, 0]
machine_old = machine_data.iloc[:, 1]

In [68]:
# Calculating mean for the old machine

mean_old = machine_old.mean()
mean_old

43.230000000000004

In [69]:
# One-tailed t-test
# H0: mean of new machine >= mean of old machine
# Ha: mean of new machine < mean of old machine

alpha = 0.05

t_stat, p_value = st.ttest_1samp(machine_new, mean_old, alternative='less')

print('t-statistic: {:.5f}'.format(t_stat))
print('p-value: {:.5f}'.format(p_value))

if p_value <= alpha:
    print('p-value <= alpha : we reject null hypothesis -> the performance of the new machine equal or better')
else:
    print('p-value > alpha : we accept null hypothesis -> the performance of the new machine is worse')

t-statistic: -5.04332
p-value: 0.00035
p-value <= alpha : we reject null hypothesis -> the performance of the new machine equal or better


Matched Pairs Test - In this challenge we will compare dependent samples of data describing our Pokemon (file files_for_lab/pokemon.csv). Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. Compare the two columns to see if there is a statistically significant difference between them and comment on your result.

In [70]:
# Reading csv as dataframe

pokemon_data = pd.read_csv('files_for_lab/pokemon.csv')
pokemon_data

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [71]:
# Slicing attack and defense data

attack = pokemon_data['Attack']
defense = pokemon_data['Defense']

In [72]:
# Matched pairs test
# H0: mean attack = mean defense -> mean attack - mean defense = 0
# Ha: mean attack != mean defense -> mean attack - mean defense != 0

alpha = 0.05 / 2 # 2-tailed

t_stat, p_value = st.ttest_rel(attack, defense)

print('t-statistic: {:.5f}'.format(t_stat))
print('p-value: {:.5f}'.format(p_value))

if p_value <= alpha:
    print('p-value <= alpha : we reject null hypothesis -> the attack and defense statistics are not equal')
else:
    print('p-value > alpha : we accept null hypothesis -> the attack and defense statistics are equal')

t-statistic: 4.32557
p-value: 0.00002
p-value <= alpha : we reject null hypothesis -> the attack and defense statistics are not equal


## ANOVA

Part 1

In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. After the next lesson, we will ask you to solve this problem using Python. Here are the steps that you would need to work on: - Null hypothesis - Alternate hypothesis - Level of significance - Test statistic - P-value - F table

Context

Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data anova_lab_data.xlsx file in the files_for_lab folder

- State the null hypothesis
- State the alternate hypothesis
- What is the significance level
- What are the degrees of freedom of the model, error terms, and total DoF

Data were collected randomly and provided to you in the table as shown: https://education-team-2020.s3-eu-west-1.amazonaws.com/data-analytics/7.05/7.05-lab_data.png


In [73]:
# Reading xlsx as dataframe

chip_data = pd.read_excel('files_for_lab/anova_lab_data.xlsx')
chip_data.columns = ['Power', 'Etching Rate']
chip_data

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


In [74]:
# Grouping by power to check mean etching rate and number of values for each power level

grouped_df = chip_data.groupby('Power')['Etching Rate'].agg(mean_etching_rate='mean', sample='size').reset_index()
grouped_df


,Power,mean_etching_rate,sample
0,160 W,5.792,5
1,180 W,6.238,5
2,200 W,8.318,5


In [75]:
# H0: mean etching rate of 160W = mean etching rate of 180W = mean etching rate of 200W
# Ha: not all means are equal

alpha = 0.05 # Significance level

In [76]:
# In our case, the groups are the Power levels, and the "y" is the "Etching Rate"

# Calculating SST
S2t = 0
for power in chip_data['Power'].unique():
    ng = len(chip_data[chip_data['Power'] == power])
    S2t  += ng * ( ( chip_data[chip_data['Power'] == power]['Etching Rate'].mean() - chip_data['Etching Rate'].mean() ) ** 2)
S2t /= ( chip_data['Power'].nunique() - 1 ) # Number of  K = groups/cities

print("The value of S2t is {:.2f}".format(S2t))

# Calculating SSE
S2E = 0
for power in chip_data['Power'].unique():
    ng = len(chip_data[chip_data['Power'] == power])
    S2E += ( ng - 1 ) * np.var(chip_data[chip_data['Power'] == power]['Etching Rate'],ddof=1)
S2E /= ( len(chip_data) - chip_data['Power'].nunique() )

print("The value of S2E is {:.2f}".format(S2E))

The value of S2t is 9.09
The value of S2E is 0.25


In [77]:
# Calculating F statistic

F = S2t / S2E
print("The value of F is {:.2f}".format(F))

The value of F is 36.88


In [78]:
# Calculating the degrees of freedom

d1 = chip_data['Power'].nunique() - 1
d2 = len(chip_data) - chip_data['Power'].nunique()

print("Number of degrees of freedom d1: ",d1)
print("Number of degrees of freedom d2: ",d2)

Number of degrees of freedom d1:  2
Number of degrees of freedom d2:  12


Part 2

In this section, use Python to conduct ANOVA.

What conclusions can you draw from the experiment and why?

In [79]:
# ANOVA with p-value

p_val = 1 - st.f.cdf(F,dfn=d1, dfd=d2)

print('p-value: {:.5f}'.format(p_val))

if p_val < alpha:
    print('The probability to get a value bigger than F is less than alpha, therefore we reject H0')
else:
    print('The probability to get a value bigger than F is greater than alpha, therefore we accept H0')

p-value: 0.00001
The probability to get a value bigger than F is less than alpha, therefore we reject H0


In [80]:
# ANOVA with critical value

Fc = st.f.ppf(1-0.05,dfn=d1, dfd=d2)

print("The critical value which corresponds to an area of 0.05 is: {:.5f}".format(Fc))

if F > Fc:
    print('F is greater than the critical value, therefore we reject H0')
else:
    print('F is smaller than the critical value, therefore we accept H0')

The critical value which corresponds to an area of 0.05 is: 3.88529
F is greater than the critical value, therefore we reject H0


In [81]:
# Getting p-val and Fc with scipy

print(st.f_oneway(chip_data[chip_data['Power'] == '160 W']['Etching Rate'],chip_data[chip_data['Power'] == '180 W']['Etching Rate'],
                  chip_data[chip_data['Power'] == '200 W']['Etching Rate']))

F_onewayResult(statistic=36.87895470100505, pvalue=7.506584272358903e-06)
